In [1]:

#%matplotlib inline 

###########
#PDF PROCESSING FOR various INPUT 
# DEPENDING ON THE PDF_PROCESSING CLASS
#by Carl Schleussner, Climate Analytics
#carl.schleussner@climateanalytics.org
###########
# IMPORT AND CONFIG 
import numpy as np
import netCDF4 as net
import dimarray as da 
import sys
sys.path.append("./src/")
import itertools
import glob
import datetime
import pickle
import os
import pandas as pd
import matplotlib.pylab as plt 
from netCDF4 import Dataset,netcdftime,num2date

#plt.style.use('ggplot')
#plt.rcParams['figure.figsize'] = 8,6
plt.rc('text', usetex=True)

os.chdir('/Users/peterpfleiderer/Documents/Projects/0p5_observed/pdf_processing-srex/')
try:
    import pdf_processing as pdf; reload(pdf)
except ImportError:
    raise ImportError(
        "cannot find PDF_Processing code")

###########
# Settings
###########

# PDF Method (currently defined: hist, python_silverman)
pdf_method='python_silverman'

# Variablelist
varinlist=[
    'TXx',
    'TNn'
          ] 

ref_period=[1979,2002]
target_periods=[[1990,2000],ref_period]
period_names=['Recent','ref']

timeaxis=np.arange(1958,2011)

# Input datasets
datasets=['era40','erainterim']

# Set plottint colours
colordict={
    datasets[0]:"#009E73",
    datasets[1]:"#663399"}


varoutdict={
    datasets[0]:{},    
    datasets[1]:{}}



In [2]:
#################
# ERA40 Erainterim overlap plot
#################


for varin in varinlist:
    fig,ax=plt.subplots(nrows=1,figsize=(4,4))
    for dataset in datasets:
        if dataset=='era40':in_file=glob.glob('/Users/peterpfleiderer/Box Sync/0p5_observational_record/data/data_climdex/*/'+varin.lower()+'*'+dataset+'*2002.nc')[0]
        if dataset=='erainterim':in_file=glob.glob('/Users/peterpfleiderer/Box Sync/0p5_observational_record/data/data_climdex/*/'+varin.lower()+'*'+dataset+'*2016.nc')[0]
        nc_in=Dataset(in_file,"r")

        # handle time information
        try:
            time=nc_in.variables['time'][:]
            datevar = []
            # if no specification
            time_unit=nc_in.variables['time'].units
            try:
                cal_temps = nc_in.variables['time'].calendar
                datevar.append(num2date(time,units = time_unit,calendar = cal_temps))
            except:
                datevar.append(num2date(time,units = time_unit))

            # create index variable
            year=np.array([int(str(date).split("-")[0])	for date in datevar[0][:]])
        except:
            time=nc_in.variables['time'][:]
            year=np.array([int(str(date)[0:4])	for date in time])


        # lon,lat
        lat=nc_in.variables['latitude'][:]
        lon=nc_in.variables['longitude'][:]

        # Set regular integer time axis, dimarray default always adds month and day. Time range 1951-2015
        relevant_years=np.where((year>=1979) & (year<2002))[0]
        var_in=nc_in.variables['t2m'][:,:,:]
        if var_in.mean()>150:var_in-=273.15

        input_data=da.DimArray(var_in[relevant_years,:,:].squeeze(), axes=[year[relevant_years], lat, lon],dims=['year', 'lat', 'lon'] )

        # load landmask
        landmask=Dataset('support/181x360_landmask.nc').variables['landmask'][:,:]
        GRL_mask=Dataset('support/GRL_181x360_lat_weighted.nc4').variables['GRL'][:,:]
        landmask[np.isfinite(GRL_mask)]=0
        polar_lats=np.where(lat<=-65)[0]
        landmask[polar_lats,:]=0
        landmask[landmask!=0]=1

        # Mask for data availability (individual for each dataset)
        varoutdict[dataset][varin]=pdf.PDF_Processing(varin)
        varoutdict[dataset][varin].mask_for_ref_period_data_coverage(input_data,ref_period,check_ref_period_only=True,landmask=landmask)

        ###########
        # Derive time slices
        ###########
        varoutdict[dataset][varin].derive_time_slices(ref_period,target_periods,period_names)
        varoutdict[dataset][varin].derive_distributions()

        dat=varoutdict[dataset][varin]._distributions['global']['ref']
        mdat = dat[np.isfinite(dat)]
        range_scaling_factor=0.5
        bin_range=[mdat.min(),mdat.max()]

        output,no_nans=varoutdict[dataset][varin].kernel_density_estimation(mdat,bin_range)

        ax.plot(output[:,0],output[:,1],color=colordict[dataset],label=dataset)
    
    
    ax.set_xlabel(varin+' [$^\circ$ C]')
    ax.legend(loc='best')
    #plt.title(varin.upper()+' 1979-2002')
    plt.savefig('../plots/ERA_'+varin+'_global_1979-2002.png')
    plt.savefig('../plots/ERA_'+varin+'_global_1979-2002.pdf')
    plt.clf()

No of non-NAN grid cells in Reference Mask:  65160
No of non-NAN grid cells in Reference Mask:  65160
No of non-NAN grid cells in Reference Mask:  65160
No of non-NAN grid cells in Reference Mask:  65160
